In [2]:
# App Libs
from gather.gather import Gather
from prof.profile import Profile
from clean.clean import Clean
from feature_engineering.engineer import Engineering

from automate.automate import Automate

from utils.utils import has_missing_data

# Generic
import random

In [3]:
# Titanic Data 
tn_filepath = '../../data/sample_data/titanic/train.csv'
tn_target = 'Survived'

# Raisin 
rs_filepath = '../../data/sample_data/raisin/Raisin_Dataset.csv'
rs_target = 'Class'

# House Prices | Regression | ???
# hp = auto_preproc('data/sample_data/house_price_data/train.csv', 'classification', 'SalePrice')

# Wasabi
# ws = auto_preproc('data/sample_data/Wasabi/batterytesters_dataset.json', 'classification', 'commonalarm')

# Credit Fraud
cc_filepath = '../../data/sample_data/credit_card_fraud/creditcard.csv'
cc_target = 'Class'

# Rain Australia
ra_filepath = '../../data/sample_data/rainAustralia/weatherAUS.csv'
ra_target = 'RainTomorrow'

# Diabetes 
db_filepath = '../../data/sample_data/diabetes/diabetes.csv'
db_target = 'Outcome'

# Heart Disease
hd_filepath = '../../data/sample_data/heart/heart.csv'
hd_target = 'target'

In [4]:
auto_params = {
    'filepath' : hd_filepath,
    'target' : hd_target,
    'problem_type' : 'classification',
    'opt_method': 'random',
    'n_iter' : 1
}
automate = Automate(auto_params)
res = automate.auto_preproc()

{'profile': {'unique_values_thres': 0.2, 'id_thres': 0.9}, 'clean': {'drop_thres': 0.7, 'outlier_thres': 2, 'num_type': 'mean'}, 'engineering': {'freq_thres': 0.07532036484579736, 'encode_type': 'label', 'scale': False, 'scale_type': 'robust', 'select': False, 'select_type': 'univariate'}, 'model': {'model': 'rf'}}
0.85070327697142
accuracy: 0.8009950248756219
precision: 0.7692307692307693
recall: 0.8737864077669902
f1: 0.8181818181818182
Iteration: 1, Accuracy: 0.8009950248756219


NameError: name 'resultp' is not defined

In [ ]:
len(automate.df[automate.df['target'] == 1])

In [ ]:
automate.df_engineered